In [8]:
# Default env type is test
# Change this when running on colab or kaggle
ENV_TYPE = "TEST"

In [ ]:
# Important deps
!apt update && apt install -y libgl1-mesa-glx imagemagick

if ENV_TYPE != "TEST":
    !apt install -y git aria2
    # Clone the repo
    !git clone https://github.com/kk-digital/kcg-ml-sd1p4.git --recurse-submodules -b lora
    %cd kcg-ml-sd1p4
    # Download model weights
    !aria2c https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -o ./input/model/sd-v1-4.ckpt -j 10

In [ ]:
# Install reqs
!pip3 install -r lora-tuning/requirements.txt

In [17]:
# Let's now run the 'test suite'
if ENV_TYPE != "TEST":
    !python3 lora-tuning/lora_train.py --num_repeats 16 --lowram
else:
    !python3 lora-tuning/lora_train.py --num_repeats 16

2023-06-23 20:34:03.430284: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 20:34:05.675802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: lora_train.py
       [-h]
       [--dataset DATASET]
       [--repo_dir REPO_DIR]
       [--activation_tags ACTIVATION_TAGS]
       [--num_repeats NUM_REPEATS]
       [--max_train_epochs MAX_TRAIN_EPOCHS]
       [--save_every_n_epochs SAVE_EVERY_N_EPOCHS]
       [--config_file CONFIG_FILE]
       [--config_dir CONFIG_DIR]
       [--dataset_config_file DATASET_CONFIG_FILE]
       [--model_file MODEL_FILE]
       [--log_dir LOG_DIR]
       [--output_dir OUTPUT_DIR]
       [--unet_lr UNET_LR]
       [--text_encoder_lr TEXT_ENCODER_LR]
       [--n

In [12]:
# Shuffle up dependencies to fix inference
!pip3 install -r lora-tuning/inference/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement triton==2.0.1 (from versions: 2.0.0.dev20221030, 2.0.0.dev20221031, 2.0.0.dev20221101, 2.0.0.dev20221103, 2.0.0.dev20221105, 2.0.0.dev20221117, 2.0.0.dev20221120, 2.0.0.dev20221202, 2.0.0.dev20230208, 2.0.0.dev20230217, 2.0.0a1, 2.0.0a2, 2.0.0, 2.0.0.post1)
ERROR: No matching distribution found for triton==2.0.1


In [14]:
# Generate images for each epoch:
import os
import subprocess

# Set the directory containing the .safetensors files
lora_path = "./output/LoRas/test/output/"
checkpoint_path = "input/model/sd-v1-4.ckpt"
scale = 512
prompt = "a sketch of a pixwaifu, cute, small, chibi, DeviantArt trending"

# Get a list of all .safetensors files in the directory
file_list = [filename for filename in os.listdir(lora_path) if filename.endswith(".safetensors")]

# Set up epoch to begin counting
epoch = 1

for filename in file_list:
    # Generate the image using txt2img.py for each .safetensors file
    output_filename = os.path.splitext(filename)[0] + ".png"
    command = f"python3 lora-tuning/inference/txt2img.py --checkpoint_path {checkpoint_path} --lora {lora_path} --output {output_filename} --scale {scale} --prompt \"{prompt}\""
    subprocess.run(command, shell=True, check=True)

    epoch += 1

# Generate a tiled image with all of the epochs
!montage -label '%t' *.png -thumbnail 512x512! -pointsize 22 -gravity center \
    -background Transparent -geometry +0+0 -tile 3x results_lora_epochs.png

FileNotFoundError: ignored